In [1]:
year_int = 2025                 #for 2025-26, year=2025
state_name = 'Tamil Nadu'       #provide state name
drive = 'F'                     #drive other than C-drive e.g. 'D', 'E', 'F'

In [14]:
import numpy as np
import pandas as pd
import pyodbc
import sys
import os
import time
from datetime import date
import re
import warnings as wr
wr.filterwarnings('ignore')

yr_int = year_int - 2001
year = str(year_int)
today = date.today().strftime("%Y.%m.%d")
states ={
    'Tamil Nadu' : 33
    ,'Puducherry' : 34
    ,'Andhra Pradesh' : 28
    ,'Telangana' : 36
    ,'Gujarat' : 24
    ,'Uttar Pradesh' : 9
    ,'Karnataka' : 29
}
state = states[state_name]

mdb_path = r"C:/ICS_Scheme/AS2.0/AS"+year+"/NSSAS"+year+".mdb"

#create connection with mdb database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    f'DBQ={mdb_path};'
)
conn = pyodbc.connect(conn_str)

#get all table names from mdb database
#### cursor = conn.cursor()
#### for row in cursor.tables(tableType='TABLE'):
####     print(row.table_name)
#### cursor.close()

#get data from the table - EXPTMASTER
sql_query = "SELECT EXPTID,SEASONCODE,SEASONNAME,SAMPLE,STATE,RONAME,DISTRICTNAME,DISTRICT,SELORDER,EXPT,CROPCODE,CROPNAME FROM EXPTMASTER"
df_ML = (
    pd
    .read_sql(sql_query, conn)
    .astype({
        'EXPTID' : 'int64',
        'SEASONCODE' : 'int16',
        'SAMPLE' : 'int16',
        'STATE' : 'int16',
        'DISTRICT' : 'int16',
        'SELORDER' : 'int16',
        'EXPT' : 'int16',
        'CROPCODE' : 'int16'
    })
)

#get data from the table - Sch20RC7
sql_query = "SELECT EXPTID,STAGIN FROM Sch20RC7"
df_sch = (
    pd
    .read_sql(sql_query, conn)
    .astype({'EXPTID' : 'int64'})
)
conn.close()

#join ML with Sch20RC7 - all states
df = pd.merge(df_ML, df_sch, how='left', left_on='EXPTID', right_on = 'EXPTID')
df['CROP'] = df['CROPCODE']//100
districts = df['DISTRICT'].unique()
ros = df['RONAME'].unique()
crops = df['CROP'].unique()
seasons = df['SEASONCODE'].unique()
samples = df['SAMPLE'].unique()

#complete ML with status of received schedules for selected state
df = df[df['STATE'] == state]
#Received Schedules ML
df_r = df[df['STAGIN'].notna()]
#Non-Receipt ML
df_nr = df[df['STAGIN'].isna()]

#NR Path
nr_path = f"{drive}:/Downloads/{today} NR AS2.0/"

#crop-wise NR
for sample in samples:
    if sample == 1:
        samp = 'Central'
    elif sample == 2:
        samp = 'State'

    for season in seasons:
        if season in (1,2,3):
            sn = 'Kharif'
        elif season in (4,5):
            sn = 'Rabi'
            

        df_temp = df_nr[(df_nr['SAMPLE'] == sample) 
                        & (df_nr['SEASONCODE'] == season)]
        df_temp.drop(columns = ['EXPTID','SEASONCODE','SAMPLE','STATE','CROPCODE','CROP','STAGIN'], inplace=True)
        if df_temp.shape[0]>0:
            os.makedirs(f"{nr_path}01 NR Complete", exist_ok=True)
            df_temp.to_csv(f"{nr_path}01 NR Complete/{samp} {sn} - Complete NR.csv", index=False)

#RO-wise NR - Central Only
for season in seasons:
    if season in (1,2,3):
        sn = 'Kharif'
    elif season in (4,5):
        sn = 'Rabi'
        
    for ro in ros:
        df_temp = df_nr[(df_nr['SAMPLE'] == 1) 
                        & (df_nr['SEASONCODE'] == season)
                        & (df_nr['RONAME'] == ro)]
        df_temp.drop(columns = ['EXPTID','SEASONCODE','SAMPLE','STATE','CROPCODE','CROP','STAGIN','RONAME'], inplace=True)
        if df_temp.shape[0]>0:
            os.makedirs(f"{nr_path}02 NR RoWise - Central", exist_ok=True)
            df_temp.to_csv(f"{nr_path}02 NR RoWise - Central/Central_{sn} - RO_{ro}.csv", index=False)

#District-wise NR - State Only
for season in seasons:
    if season in (1,2,3):
        sn = 'Kharif'
    elif season in (4,5):
        sn = 'Rabi'
        
    df_temp = df_nr[(df_nr['SAMPLE'] == 2) & (df_nr['SEASONCODE'] == season)]
    df_temp.drop(columns = ['EXPTID','SEASONCODE','SAMPLE','STATE','CROPCODE','CROP','STAGIN','RONAME'], inplace=True)
    if df_temp.shape[0]>0:
        os.makedirs(f"{nr_path}03 NR DisttWise - State", exist_ok=True)
        df_temp.to_csv(f"{nr_path}03 NR DisttWise - State/State_{sn} - NR DistrictWise.csv", index=False)